In [1]:
from MNIST_dataloader import Noisy_MNIST
from Fast_MRI_dataloader import Fast_MRI
import matplotlib.pyplot as plt
import torch.nn as nn
import torch
import torch.optim as optim
from torch.utils.data import Dataset,DataLoader
from torchvision import transforms
from torch.utils.data import TensorDataset
import glob
import numpy as np
from tqdm.auto import tqdm 
from PIL import Image
import numpy as np
from torch.fft import fft2, fftshift, ifft2, ifftshift

c:\Users\amin2\anaconda3\envs\MachineLearningSignalProcessing\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# %% dataloader for the Fast MRI dataset
def create_dataloaders_mri(data_loc, batch_size):
    dataset_train = Fast_MRI("train", data_loc)
    dataset_test  = Fast_MRI("test" , data_loc)
    
    Fast_MRI_train_loader =  DataLoader(dataset_train, batch_size=batch_size, shuffle=True,  drop_last=False)
    Fast_MRI_test_loader  =  DataLoader(dataset_test , batch_size=batch_size, shuffle=True, drop_last=False)
    
    return Fast_MRI_train_loader, Fast_MRI_test_loader

In [3]:
# Bram
#data_loc = 'D://5LSL0-Datasets//Fast_MRI_Knee' #change the datalocation to something that works for you
#Amin 
data_loc = 'C:/Users/amin2/Documents/School/5LSL0ML/5LSL0/data/Fast_MRI_Knee'


# define parameters
batch_size = 8

train_loader, test_loader = create_dataloaders_mri(data_loc, batch_size)

In [5]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self, ).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(16)
        self.relu1 = nn.LeakyReLU()
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.deconv2 = nn.ConvTranspose2d(in_channels=32, out_channels=16, kernel_size=3, stride=1, padding=1)
        self.deconv1 = nn.ConvTranspose2d(in_channels=16, out_channels=1, kernel_size=3, stride=1, padding=1)


    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu1(x)
        x = self.conv2(x)
        x = self.deconv2(x)
        x = self.deconv1(x)
        return x

In [6]:
# Define function to achieve Full k-space
def get_k_space(inputs):
    k_space = fftshift(fft2(inputs))
    return k_space

# Define function to achieve Partial k-space from Full k-space and Mask
def get_partial_k_space(input,M):
    return  torch.mul(input, M)

def get_accelerate_MRI(inputs):
    return ifft2(inputs)

In [11]:


criterion = nn.MSELoss()
num_epochs = 10
#unfolding steps
num_iterations = 5
model1 = ConvNet()
model2 = ConvNet()
model3 = ConvNet()
model4 = ConvNet()
model5 = ConvNet()

optimizer1 = optim.Adam( model1.parameters(), lr=0.001)
optimizer2 = optim.Adam( model2.parameters(), lr=0.001)
optimizer3 = optim.Adam( model3.parameters(), lr=0.001)
optimizer4 = optim.Adam( model4.parameters(), lr=0.001)
optimizer5 = optim.Adam( model5.parameters(), lr=0.001)
mu = 0.1

0
1
2
3
4


In [20]:
for epoch in range(num_epochs):
    train_loss = 0.0
    count = 0

    # Training loop
    loop = tqdm(train_loader)
    loop.set_description(f"Epoch [{epoch}/{num_epochs}]")
    for i,(kspace, M, gt) in enumerate(loop):
        gt_label = gt.unsqueeze(1)
        kspace_input = kspace.unsqueeze(1)
        acc_mri = torch.abs(ifft2(kspace_input))
        x_t = acc_mri

        for iter in range(num_iterations):

            if(iter == 0):
                model1.train()
                optimizer1.zero_grad()
                x_t = model1(x_t)
            elif(iter == 1):
                model2.train()
                optimizer2.zero_grad()
                x_t = model2(x_t)
            elif(iter == 2):
                model3.train()
                optimizer3.zero_grad()
                x_t = model3(x_t)
            elif(iter == 3):
                model4.train()
                optimizer4.zero_grad()
                x_t = model4(x_t)
            elif(iter == 4):
                model5.train()
                optimizer5.zero_grad()
                x_t = model5(x_t)
            F_x = get_k_space(x_t)
            k_space_y = get_k_space(acc_mri)
            F_x = torch.squeeze(F_x, dim=1) 
            k_space_y = torch.squeeze(k_space_y, dim=1) 
            z = F_x - mu * get_partial_k_space(F_x, M) + mu * get_partial_k_space(k_space_y, M)
            z = torch.unsqueeze(z, dim=1) 
            x_t = torch.abs(get_accelerate_MRI(z))


        loss = criterion(x_t, gt_label)
        loss.backward()
        optimizer1.step()
        optimizer2.step()
        optimizer3.step()
        optimizer4.step()
        optimizer5.step()
        count+=1
        train_loss += loss.item() #* kspace_input.size(0)
        loop.set_postfix(loss=loss.item())
    count_test = 0
    test_loss = 0
    # Testing loop
    with torch.no_grad():
       loop = tqdm(test_loader)
       for i,(kspace_t, M_t, gt_t) in enumerate(loop):
            gt_label_t = gt_t.unsqueeze(1)
            kspace_input_t = kspace_t.unsqueeze(1)
            acc_mri_t = torch.abs(ifft2(kspace_input_t))
            x_t = acc_mri_t

            for iter in range(num_iterations):
                if(iter == 0):
                    model1.eval()
                    x_t = model1(x_t)
                elif(iter == 1):
                    model2.eval()
                    x_t = model2(x_t)
                elif(iter == 2):
                    model3.eval()
                    x_t = model3(x_t)
                elif(iter == 3):
                    model4.eval()
                    x_t = model4(x_t)
                elif(iter == 4):
                    model5.eval()
                    x_t = model5(x_t)
                F_x = get_k_space(x_t)

                k_space_y = get_k_space(acc_mri_t)
                F_x = torch.squeeze(F_x, dim=1) 
                k_space_y = torch.squeeze(k_space_y, dim=1) 
                z = F_x - mu * get_partial_k_space(F_x, M) + mu * get_partial_k_space(k_space_y, M)
                z = torch.unsqueeze(z, dim=1) 
                x_t = torch.abs(get_accelerate_MRI(z))
            loss = criterion(x_t, gt_label_t)
            count_test +=1
            test_loss += loss.item() #* kspace_input.size(0)
            loop.set_postfix(loss=loss.item())

    # Calculate average losses
    train_loss /= count
    test_loss /= count_test

    # train_losses.append(train_loss)
    # test_losses.append(test_loss)

    # Print epoch-wise loss
    print(f"Epoch {epoch+1}: Train Loss = {train_loss:.4f}, Test Loss = {test_loss:.4f}")



Epoch [0/10]:   0%|          | 0/94 [00:01<?, ?it/s]


RuntimeError: one of the variables needed for gradient computation has been modified by an inplace operation: [torch.FloatTensor [16, 1, 3, 3]] is at version 6; expected version 5 instead. Hint: enable anomaly detection to find the operation that failed to compute its gradient, with torch.autograd.set_detect_anomaly(True).